Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print "Training set", train_dataset.shape, train_labels.shape
    print "Validation set", valid_dataset.shape, valid_labels.shape
    print "Test set", test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print "Training set", train_dataset.shape, train_labels.shape
print "Validation set", valid_dataset.shape, valid_labels.shape
print "Test set", test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression with L2 Regularization

In [5]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
learning_rate = 0.5
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    
    #Regularization
    regularizers = tf.nn.l2_loss(weights)
    #loss += beta * regularizers
    loss = tf.reduce_mean(loss + (beta * regularizers))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 500 == 0):
            print "Loss at step %d: %f" % (step, l)
            print "Training accuracy: %.1f%%" % accuracy(predictions, train_labels[:train_subset, :])
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Loss at step 0: 20.588560
Training accuracy: 12.0%
Validation accuracy: 17.6%
Loss at step 500: 2.648959
Training accuracy: 78.9%
Validation accuracy: 75.8%
Loss at step 1000: 1.580140
Training accuracy: 82.3%
Validation accuracy: 77.6%
Loss at step 1500: 1.076006
Training accuracy: 84.8%
Validation accuracy: 78.9%
Loss at step 2000: 0.820536
Training accuracy: 86.0%
Validation accuracy: 80.0%
Loss at step 2500: 0.685447
Training accuracy: 86.8%
Validation accuracy: 80.5%
Loss at step 3000: 0.612025
Training accuracy: 87.2%
Validation accuracy: 80.8%
Test accuracy: 87.9%


### Logistic Regression with L2 Regularization using SGD

In [7]:
batch_size = 128
learning_rate = 0.5
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    
    #Regularization
    regularizers = tf.nn.l2_loss(weights)
    #loss = beta * regularizers
    loss = tf.reduce_mean(loss + (beta * regularizers))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 18.598919
Minibatch accuracy: 12.5%
Validation accuracy: 13.1%
Minibatch loss at step 500: 2.450012
Minibatch accuracy: 82.0%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 2.210392
Minibatch accuracy: 66.4%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 1.501318
Minibatch accuracy: 76.6%
Validation accuracy: 80.4%
Minibatch loss at step 2000: 1.017446
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 0.544912
Minibatch accuracy: 87.5%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 0.729480
Minibatch accuracy: 82.8%
Validation accuracy: 81.4%
Test accuracy: 88.3%


### Nerual Network with L2 Regularization

In [9]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layer
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    
    #Regularization
    regularizers = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + (beta * regularizers))
    #loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights) + biases)
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights) + biases)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 7.783441
Minibatch accuracy: 8.6%
Validation accuracy: 36.8%
Minibatch loss at step 500: 2.464604
Minibatch accuracy: 85.9%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 1.857801
Minibatch accuracy: 75.8%
Validation accuracy: 86.4%
Minibatch loss at step 1500: 1.262220
Minibatch accuracy: 82.8%
Validation accuracy: 88.0%
Minibatch loss at step 2000: 0.930780
Minibatch accuracy: 85.9%
Validation accuracy: 87.9%
Minibatch loss at step 2500: 0.522639
Minibatch accuracy: 95.3%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 0.646537
Minibatch accuracy: 89.1%
Validation accuracy: 88.1%
Test accuracy: 93.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

### SGD Overfitting

In [11]:
batch_size = 128
learning_rate = 0.5
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #Regularization
    #regularizers = tf.nn.l2_loss(weights)
    #loss += beta * regularizers
    #loss = tf.reduce_mean(loss + (beta * regularizers))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [12]:
num_steps = 101
num_batches = 3

#train_subset = train_dataset[:500, :]
#train_sublabels = train_labels[:500, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        #offset = (step * batch_size) % (train_sublabels.shape[0] - batch_size)
        offset = step % num_batches
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 5 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 18.313494
Minibatch accuracy: 10.2%
Validation accuracy: 13.4%
Minibatch loss at step 5: 7.324716
Minibatch accuracy: 32.0%
Validation accuracy: 26.3%
Minibatch loss at step 10: 3.926522
Minibatch accuracy: 52.3%
Validation accuracy: 35.2%
Minibatch loss at step 15: 2.513965
Minibatch accuracy: 64.1%
Validation accuracy: 41.8%
Minibatch loss at step 20: 1.367496
Minibatch accuracy: 69.5%
Validation accuracy: 44.6%
Minibatch loss at step 25: 0.717137
Minibatch accuracy: 84.4%
Validation accuracy: 46.1%
Minibatch loss at step 30: 0.613701
Minibatch accuracy: 92.2%
Validation accuracy: 47.3%
Minibatch loss at step 35: 0.317340
Minibatch accuracy: 95.3%
Validation accuracy: 48.0%
Minibatch loss at step 40: 0.162033
Minibatch accuracy: 96.1%
Validation accuracy: 48.6%
Minibatch loss at step 45: 0.275547
Minibatch accuracy: 98.4%
Validation accuracy: 49.3%
Minibatch loss at step 50: 0.114168
Minibatch accuracy: 99.2%
Validation accuracy: 49.9%
Minibatch 

### Neural Network Overfitting

In [13]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layer
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #Regularization
    #regularizers = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights)
    #loss += beta * regularizers
    #loss = tf.reduce_mean(loss + (beta * regularizers))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights) + biases)
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights) + biases)

In [14]:
num_steps = 101
num_batches = 3

#train_subset = train_dataset[:500, :]
#train_sublabels = train_labels[:500, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        #offset = (step * batch_size) % (train_sublabels.shape[0] - batch_size)
        offset = step % num_batches
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 5 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 4.855344
Minibatch accuracy: 8.6%
Validation accuracy: 29.9%
Minibatch loss at step 5: 0.396571
Minibatch accuracy: 88.3%
Validation accuracy: 67.4%
Minibatch loss at step 10: 0.016619
Minibatch accuracy: 100.0%
Validation accuracy: 69.8%
Minibatch loss at step 15: 0.009688
Minibatch accuracy: 100.0%
Validation accuracy: 70.2%
Minibatch loss at step 20: 0.006627
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Minibatch loss at step 25: 0.005250
Minibatch accuracy: 100.0%
Validation accuracy: 70.5%
Minibatch loss at step 30: 0.004636
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 35: 0.003823
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 40: 0.003344
Minibatch accuracy: 100.0%
Validation accuracy: 70.6%
Minibatch loss at step 45: 0.003153
Minibatch accuracy: 100.0%
Validation accuracy: 70.7%
Minibatch loss at step 50: 0.002752
Minibatch accuracy: 100.0%
Validation accuracy: 70.7%
Min

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Neural Network Overfitting with Dropout

In [15]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5
beta = 0.001
dropout_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layer with dropout
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
    #hidden_weights = tf.get_variable("hidden_weights", shape=[image_size * image_size, hidden_nodes], 
    #                                 initializer=tf.contrib.layers.xavier_initializer())
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    hidden_layer = tf.nn.dropout(hidden_layer, dropout_prob)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #Regularization
    #regularizers = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) + tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    #loss += beta * regularizers
    #loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights) + biases)
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights) + biases)

In [16]:
num_steps = 101
num_batches = 3

#train_subset = train_dataset[:500, :]
#train_sublabels = train_labels[:500, :]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        #offset = (step * batch_size) % (train_sublabels.shape[0] - batch_size)
        offset = step % num_batches
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 5 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 5.869611
Minibatch accuracy: 7.0%
Validation accuracy: 32.8%
Minibatch loss at step 5: 0.362991
Minibatch accuracy: 86.7%
Validation accuracy: 67.5%
Minibatch loss at step 10: 0.014627
Minibatch accuracy: 100.0%
Validation accuracy: 69.8%
Minibatch loss at step 15: 0.009160
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Minibatch loss at step 20: 0.011645
Minibatch accuracy: 99.2%
Validation accuracy: 70.4%
Minibatch loss at step 25: 0.010508
Minibatch accuracy: 100.0%
Validation accuracy: 70.8%
Minibatch loss at step 30: 0.011586
Minibatch accuracy: 100.0%
Validation accuracy: 71.0%
Minibatch loss at step 35: 0.022812
Minibatch accuracy: 99.2%
Validation accuracy: 70.4%
Minibatch loss at step 40: 0.003432
Minibatch accuracy: 100.0%
Validation accuracy: 70.4%
Minibatch loss at step 45: 0.008401
Minibatch accuracy: 100.0%
Validation accuracy: 71.0%
Minibatch loss at step 50: 0.004394
Minibatch accuracy: 100.0%
Validation accuracy: 70.9%
Minib

### Neural Network with Dropout

In [17]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5
#beta = 0.001
dropout_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layer with dropout
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes], stddev=0.1))
    #hidden_weights = tf.get_variable("hidden_weights", shape=[image_size * image_size, hidden_nodes], 
    #                                 initializer=tf.contrib.layers.xavier_initializer())
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    hidden_layer = tf.nn.dropout(hidden_layer, dropout_prob)

    # Variables.
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels], stddev=0.1))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    #Regularization
    #regularizers = tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(hidden_biases) + tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    #loss += beta * regularizers
    #loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden, weights) + biases)
    test_hidden = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden, weights) + biases)

In [18]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 5.378441
Minibatch accuracy: 10.9%
Validation accuracy: 35.6%
Minibatch loss at step 500: 0.713845
Minibatch accuracy: 84.4%
Validation accuracy: 84.6%
Minibatch loss at step 1000: 0.744019
Minibatch accuracy: 75.0%
Validation accuracy: 84.6%
Minibatch loss at step 1500: 0.783527
Minibatch accuracy: 80.5%
Validation accuracy: 86.6%
Minibatch loss at step 2000: 0.544021
Minibatch accuracy: 81.2%
Validation accuracy: 86.5%
Minibatch loss at step 2500: 0.253144
Minibatch accuracy: 93.0%
Validation accuracy: 87.5%
Minibatch loss at step 3000: 0.478455
Minibatch accuracy: 85.2%
Validation accuracy: 87.4%
Test accuracy: 93.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### Two Hidden Layer Neural Network with L2 Regularization

In [19]:
batch_size = 128
hidden_nodes1 = 1024
hidden_nodes2 = 100
initial_learning_rate = 0.5
beta = 0.001
#dropout_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    global_step = tf.Variable(0)
    
    # hidden layer 1
    hidden_weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1], 
                                                      stddev=np.sqrt(2.0 / (image_size * image_size))))
    hidden_biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1)
    
    # hidden layer 2
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], 
                                                      stddev=np.sqrt(2.0 / hidden_nodes1)))
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1, hidden_weights2) + hidden_biases2)
    
    # output layer
    out_weights = tf.Variable(tf.truncated_normal([hidden_nodes2, num_labels], 
                                                  stddev=np.sqrt(2.0 / hidden_nodes2)))
    out_biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer2, out_weights) + out_biases
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    
    #Regularization
    regularizers = tf.nn.l2_loss(hidden_weights1) + tf.nn.l2_loss(hidden_weights2) + tf.nn.l2_loss(out_weights)
    #loss += beta * regularizers
    loss = tf.reduce_mean(loss + (beta * regularizers))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights1) + hidden_biases1)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, hidden_weights2) + hidden_biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden2, out_weights) + out_biases)
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, hidden_weights2) + hidden_biases2)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden2, out_weights) + out_biases)

In [20]:
num_steps = 9001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 3.257273
Minibatch accuracy: 9.4%
Validation accuracy: 38.1%
Minibatch loss at step 500: 1.079958
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 1000: 1.058684
Minibatch accuracy: 78.9%
Validation accuracy: 87.3%
Minibatch loss at step 1500: 0.750824
Minibatch accuracy: 84.4%
Validation accuracy: 88.3%
Minibatch loss at step 2000: 0.623020
Minibatch accuracy: 89.1%
Validation accuracy: 88.5%
Minibatch loss at step 2500: 0.352005
Minibatch accuracy: 96.1%
Validation accuracy: 89.0%
Minibatch loss at step 3000: 0.531911
Minibatch accuracy: 89.1%
Validation accuracy: 89.4%
Minibatch loss at step 3500: 0.541526
Minibatch accuracy: 89.1%
Validation accuracy: 89.4%
Minibatch loss at step 4000: 0.463656
Minibatch accuracy: 89.8%
Validation accuracy: 89.8%
Minibatch loss at step 4500: 0.441993
Minibatch accuracy: 93.0%
Validation accuracy: 89.8%
Minibatch loss at step 5000: 0.467144
Minibatch accuracy: 91.4%
Validation accuracy

In [32]:
batch_size = 256
hidden_nodes1 = 1024
hidden_nodes2 = 256
hidden_nodes3 = 128
beta = 1.5e-4
initial_learning_rate = 0.5
#dropout_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    
    # hidden layer 1
    hidden_weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes1], 
                                                      stddev=np.sqrt(2.0 / (image_size * image_size))))
    hidden_biases1 = tf.Variable(tf.zeros([hidden_nodes1]))
    hidden_layer1 = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights1) + hidden_biases1)
    #hidden_layer1 = tf.nn.dropout(hidden_layer1, dropout_prob)
    
    # hidden layer 2
    hidden_weights2 = tf.Variable(tf.truncated_normal([hidden_nodes1, hidden_nodes2], \
                                                      stddev=np.sqrt(2.0 / hidden_nodes1)))
    hidden_biases2 = tf.Variable(tf.zeros([hidden_nodes2]))
    hidden_layer2 = tf.nn.relu(tf.matmul(hidden_layer1, hidden_weights2) + hidden_biases2)
    #hidden_layer2 = tf.nn.dropout(hidden_layer2, dropout_prob)
    
    # hidden layer 3
    hidden_weights3 = tf.Variable(tf.truncated_normal([hidden_nodes2, hidden_nodes3], \
                                                      stddev=np.sqrt(2.0 / hidden_nodes2)))
    hidden_biases3 = tf.Variable(tf.zeros([hidden_nodes3]))
    hidden_layer3 = tf.nn.relu(tf.matmul(hidden_layer2, hidden_weights3) + hidden_biases3)
    #hidden_layer3 = tf.nn.dropout(hidden_layer3, dropout_prob)
    
    # output layer
    out_weights = tf.Variable(tf.truncated_normal([hidden_nodes3, num_labels], \
                                                  stddev=np.sqrt(2.0 / hidden_nodes3)))
    out_biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden_layer3, out_weights) + out_biases
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    
    # Regularization
    regularizers = tf.nn.l2_loss(hidden_weights1) + tf.nn.l2_loss(hidden_weights2) + \
                    tf.nn.l2_loss(hidden_weights3) + tf.nn.l2_loss(out_weights)
    # loss += beta * regularizers
    loss = tf.reduce_mean(loss + (beta * regularizers))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_hidden1 = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights1) + hidden_biases1)
    valid_hidden2 = tf.nn.relu(tf.matmul(valid_hidden1, hidden_weights2) + hidden_biases2)
    valid_hidden3 = tf.nn.relu(tf.matmul(valid_hidden2, hidden_weights3) + hidden_biases3)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden3, out_weights) + out_biases)
    test_hidden1 = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights1) + hidden_biases1)
    test_hidden2 = tf.nn.relu(tf.matmul(test_hidden1, hidden_weights2) + hidden_biases2)
    test_hidden3 = tf.nn.relu(tf.matmul(test_hidden2, hidden_weights3) + hidden_biases3)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden3, out_weights) + out_biases)

In [33]:
num_steps = 18001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print "Initialized"
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print "Minibatch loss at step %d: %f" % (step, l)
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
    print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0: 2.577260
Minibatch accuracy: 5.9%
Validation accuracy: 31.7%
Minibatch loss at step 500: 0.643216
Minibatch accuracy: 83.2%
Validation accuracy: 87.0%
Minibatch loss at step 1000: 0.443891
Minibatch accuracy: 90.6%
Validation accuracy: 88.7%
Minibatch loss at step 1500: 0.477481
Minibatch accuracy: 89.8%
Validation accuracy: 89.5%
Minibatch loss at step 2000: 0.474967
Minibatch accuracy: 89.5%
Validation accuracy: 90.0%
Minibatch loss at step 2500: 0.373523
Minibatch accuracy: 93.4%
Validation accuracy: 90.5%
Minibatch loss at step 3000: 0.410788
Minibatch accuracy: 93.0%
Validation accuracy: 90.5%
Minibatch loss at step 3500: 0.343864
Minibatch accuracy: 94.5%
Validation accuracy: 90.8%
Minibatch loss at step 4000: 0.366202
Minibatch accuracy: 93.0%
Validation accuracy: 90.8%
Minibatch loss at step 4500: 0.278399
Minibatch accuracy: 95.7%
Validation accuracy: 91.2%
Minibatch loss at step 5000: 0.291169
Minibatch accuracy: 96.5%
Validation accuracy